last edited by Claire Valva on September 5, 2018
# Functions to be used in spectral analysis

These are functions to be used in later spectral analysis, first written in a jupyter notebook for readability (and then the last line will convert to a python script to use as a source in other files). 

In [1]:
#import packages
import numpy as np
from scipy.signal import get_window, csd
from scipy.fftpack import fft, ifft, fftfreq, fftshift, ifftshift
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import matplotlib.cm as cm
from math import pi
import matplotlib.ticker as tck
import datetime
from sympy import solve, Poly, Eq, Function, exp, re, im
from PyEMD import EMD
from netCDF4 import Dataset, num2date # This is to read .nc files and time array
from scipy.optimize import fsolve

In [2]:
#append season to each row
def season_sort(x):
    if x < 3:
        return("winter")
    elif x >= 12:
        return("winter")
    elif x >= 3 and x < 6:
        return("spring")
    elif x >= 6 and x < 9:
        return("summer")
    elif x >= 9 and x < 12:
        return("fall")
    else:
        return("error?")

In [3]:
def query_string(yearwanted, seasonwanted,lonwanted):
    if seasonwanted != "winter":
        stuff_in_string = "season == '{}' & year == {} & lon == {}".format(
            seasonwanted, yearwanted, lonwanted)
    else:
        stuff_in_string = "lon == {} and ((year == {} and month == 12) or (year == {} and (month == 1 | month == 2)))".format(
        lonwanted, yearwanted + 1, yearwanted)
    
    return stuff_in_string

In [4]:
def geopot_fft(geopotential):
    y = fft(geopotential)
    ck = y
    return(ck)

In [5]:
zonal_spacing = fftfreq(240,1.5)
zonal_spacing = 1/zonal_spacing
zonal_spacing= 360 / zonal_spacing

/home/clairev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [6]:
def solve_f(X):
    #function to solve f coeff equation for trend analysis
    x,y = X
    f = Zofkt - x*np.exp(1j*y)
    return [np.real(f), np.imag(f)] 

In [7]:
def real_f(X):
    #function to wrap solve_f so that it can be used with fsolve
    x,y = X
    z = [x+0j,y+0j]
    actual_f = solve_f(z)
    return(actual_f)

In [8]:
def fwithZ(time,wavenumber, fft_zonal_result):
    global Zofkt 
    Zofkt = fft_zonal_result[time][wavenumber]
    answers = fsolve(real_f, np.array([0,0]))
    return answers

In [9]:
def arrayforsub(a,b,N):
    #get the correct length 
    Narray = np.linspace(0, N, N)
    tosub = a*Narray + b
    return(tosub)

In [10]:
def gettimeints(timechange):
    hours = timechange.days*24 + timechange.seconds // 3600
    timeints = hours // 6
    return timeints 

In [11]:
def gettimes(frame, tarray):
    toplot_df = frame
    index_df = list(toplot_df.index)
    
    from_start = list(toplot_df["datetime"])[0] - tarray[0]
    timechange = list(toplot_df["datetime"])[len(index_df)-1] - list(toplot_df["datetime"])[0]
    
    time_ints_fromstart = gettimeints(from_start)
    time_ints_mid = gettimeints(timechange)
    
    return time_ints_fromstart, time_ints_mid

In [12]:
def trendsolver(wavenumber, frame, tarray, fft_zonal_result):
    #get first and last timestamp, EDIT
    tmin, tmax = gettimes(frame, tarray)
    tmax = tmin + tmax
    
    min_answers = fwithZ(tmin, wavenumber, fft_zonal_result)
    max_answers = fwithZ(tmax, wavenumber, fft_zonal_result)
    
    #solve for slope
    a_top = max_answers[0] - min_answers[0]
    t_diff = tmax - tmin
    a = a_top/t_diff
    
    #solve for intercept
    b = -a*(t_diff)/2
    
    #get the array to later subtract
    array = arrayforsub(a,b,t_diff)
    return(array)

In [13]:
def pairwavenum(i, frame, tarray, fft_zonal_result):
    #pairs the wavenumber with the trend array
    trendarray = trendsolver(i, frame, tarray, fft_zonal_result)
    wavenumber = zonal_spacing[i]
    
    return(wavenumber,trendarray)

In [14]:
def coeff_subtract(time, frame, tarray, fft_zonal_result):
    #get first and last timestamp
    tmin, tmax = gettimes(frame, tarray)
    
    #get the time steps at the beginning and end of the seasons
    iter_list = time - tmin
    
    #get the results and then subtract the list
    results = fft_zonal_result[time]
    tosub = [sublist[1][iter_list] for sublist in trend_list]
    
    results = results - tosub
    
    return results

In [18]:
def query_string_v2(yearwanted, seasonwanted,lonwanted):
    stuff_in_string = "season == '{}' & year == {} & lon == {}".format(
        seasonwanted, yearwanted, lonwanted)
    
    return stuff_in_string

In [20]:
!jupyter nbconvert --to=script functions_forspectralanalysis.ipynb

[NbConvertApp] Converting notebook functions_forspectralanalysis.ipynb to script
[NbConvertApp] Writing 4656 bytes to functions_forspectralanalysis.py
